In [ ]:
sm = snakemake

In [ ]:
import spherpro.bro as spb
import spherpro.db as db

import pandas as pd
import pathlib
import sqlalchemy as sa

## Remove spheres determined bad in the BF analysis

Applies the QC analysis from:
https://github.com/BodenmillerGroup/SpheroidPaper/tree/master/analysis/20190311_oexp_bfanalysis-bigexperiment2_v1

In [ ]:
class C:
    fn_config = sm.input.fn_config 
    fn_bf_qc = sm.input.fn_bf_qc



In [ ]:
bro = spb.get_bro(C.fn_config)

In [ ]:
n_cond = (bro.session.query(db.conditions.condition_id)
            .join(db.images)
             .join(db.valid_images)
              .group_by(db.conditions.condition_id).count())
n_img = bro.session.query(db.valid_images.image_id).count()
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
            .filter(db.objects.object_type == 'cell').count())




print(n_cond,n_img, n_obj)

Load data

In [ ]:
dat_bfqc = pd.read_csv(C.fn_bf_qc)

In [ ]:
list(dat_bfqc.columns)

In [ ]:
bfqc_good = 'good'
bfqc_colqc ='bf_qc'
bfqc_well = 'well'
bfqc_plate = 'plate'

In [ ]:
dat_bfqc_exclude = dat_bfqc.query(f'{bfqc_colqc} != "{bfqc_good}"')

Get imageids to delete

In [ ]:
del_tups = list(dat_bfqc_exclude[[bfqc_plate, bfqc_well]].itertuples(index=False))
bfbad_imids = [i[0] for i in (bro.session.query(db.images.image_id)
 .join(db.valid_images)
 .join(db.conditions)
  .filter(sa.tuple_(db.conditions.plate_id, db.conditions.well_name).in_(del_tups))
 ).all()]

In [ ]:
n_cond = (bro.session.query(db.images.condition_id)\
                                     .filter(db.images.image_id.in_(bfbad_imids))
                                      .group_by(db.images.condition_id)
                                     .count())

In [ ]:
print(f'{len(bfbad_imids)} images of {n_cond} conditions removed via brightfield qc')

Remove the images

In [ ]:
stmt_bfbad = (bro.session.query(db.valid_images)
             .filter(db.valid_images.image_id.in_(bfbad_imids)))

In [ ]:
stmt_bfbad.delete(synchronize_session='fetch')

bro.session.commit()

Small sanity checks what is left

In [ ]:
q = (bro.session.query(db.images, db.conditions)
         .join(db.conditions)
         .join(db.valid_images)
    )
d = bro.doquery(q)



In [ ]:
d.shape

In [ ]:
d[db.conditions.condition_id.key].nunique()

In [ ]:
q = (bro.session.query(db.valid_images)
    )
d = bro.doquery(q)

In [ ]:
n_cond = (bro.session.query(db.images.condition_id)
             .join(db.valid_images)
              .group_by(db.images.condition_id).count())
n_img = bro.session.query(db.valid_images.image_id).count()
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
            .filter(db.objects.object_type == 'cell').count())




print(n_cond,n_img, n_obj)

Objects within valid images

In [ ]:
n_obj = (bro.session.query(db.valid_objects.object_id)
            .join(db.objects)
             .join(db.images)
             .join(db.valid_images)
            .filter(db.objects.object_type == 'cell').count())

In [ ]:
n_obj

In [ ]:
n_obj

In [ ]:
n_obj